In [5]:
%load_ext lab_black
import subprocess, os

In [3]:
out_dir = "/Extension_HDD1/Xen_PP/data/cXen_FoxA2_CR_rep2/"
peak_summit = "/Extension_HDD1/Xen_PP/data/cXen_FoxA2_CR_rep2/cXen_FoxA2_CR_rep2_0_120.bowtie_tune.macs2_summits.bed"
mask_genome_fa = "/Extension_HDD1/mouse_embryo_sc_rna_seq/mESC_PrE/cutandrun_test/hts.igb.uci.edu/paulap21071953/mm10.fa"
peak_size = 300
out_prefix = out_dir + peak_summit.split("/")[-1]

# Remove Peaks with Repeats

Sequence of padded peaks

In [7]:
padded_peaks = f"{out_prefix}.padded_{peak_size}.bed"
with open(padded_peaks, "w") as o:
    subprocess.run(
        f"bedops --range {int(peak_size / 2)} -u {peak_summit}", shell=True, stdout=o
    )

padded_peaks_fa = f"{out_prefix}.padded_{peak_size}.fa"
subprocess.run(
    f"bedtools getfasta -fi {mask_genome_fa} -bed {padded_peaks} -fo {padded_peaks_fa}",
    shell=True,
)

Feature (chrUn_JH584304:114237-114538) beyond the length of chrUn_JH584304 size (114452 bp).  Skipping.


CompletedProcess(args='bedtools getfasta -fi /Extension_HDD1/mouse_embryo_sc_rna_seq/mESC_PrE/cutandrun_test/hts.igb.uci.edu/paulap21071953/mm10.fa -bed /Extension_HDD1/Xen_PP/data/cXen_FoxA2_CR_rep2/cXen_FoxA2_CR_rep2_0_120.bowtie_tune.macs2_summits.bed.padded_300.bed -fo /Extension_HDD1/Xen_PP/data/cXen_FoxA2_CR_rep2/cXen_FoxA2_CR_rep2_0_120.bowtie_tune.macs2_summits.bed.padded_300.fa', returncode=0)

Remove repeats

In [2]:
# using pipeline from: https://bitbucket.org/qzhudfci/cutruntools/src/master/
# "CUT&RUNTools: a flexible pipeline for CUT&RUN processing and footprint analysis", Qian Zhu et al, 2019
filter_repeat_script = "/Extension_HDD1/mouse_embryo_sc_rna_seq/mESC_PrE/cutandrun_test/hts.igb.uci.edu/paulap21071953/cutruntools/macs2.narrow.aug18/filter.py"

In [10]:
peaks_has_repeats = f"{out_prefix}.repeat_summits.bed"
with open(peaks_has_repeats, "w") as o:
    subprocess.run(
        f"{filter_repeat_script} {padded_peaks_fa} {peak_size}", shell=True, stdout=o
    )

filtered_peaks_summits = f"{out_prefix}.filtered_summits.bed"
with open(filtered_peaks_summits, "w") as o:
    subprocess.run(
        f"bedops -n 1 {peak_summit} {peaks_has_repeats}", shell=True, stdout=o
    )

Sequence of filtered peaks

In [11]:
padding_proc = subprocess.Popen(
    f"bedops --range {int(peak_size / 2)} -u {filtered_peaks_summits}",
    shell=True,
    stdout=subprocess.PIPE,
)

filtered_padded_peaks_fa = f"{out_prefix}.filtered_summits.padded_{peak_size}.fa"
subprocess.run(
    f"bedtools getfasta -fi {mask_genome_fa} -bed - -fo {filtered_padded_peaks_fa}",
    shell=True,
    stdin=padding_proc.stdout,
)

Feature (chrUn_JH584304:114237-114538) beyond the length of chrUn_JH584304 size (114452 bp).  Skipping.


CompletedProcess(args='bedtools getfasta -fi /Extension_HDD1/mouse_embryo_sc_rna_seq/mESC_PrE/cutandrun_test/hts.igb.uci.edu/paulap21071953/mm10.fa -bed - -fo /Extension_HDD1/Xen_PP/data/cXen_FoxA2_CR_rep2/cXen_FoxA2_CR_rep2_0_120.bowtie_tune.macs2_summits.bed.filtered_summits.padded_300.fa', returncode=0)

# Run Meme

In [12]:
sample_n_peaks = 5000
meme_outdir = f"{out_prefix}.meme_{sample_n_peaks}_sample/"
subprocess.run(
    f"meme-chip -oc {meme_outdir} -streme-nmotifs 20 -meme-nmotifs 20 {filtered_padded_peaks_fa} -meme-p 10 -meme-searchsize {sample_n_peaks * peak_size}",
    shell=True,
)

Starting getsize: getsize /Extension_HDD1/Xen_PP/data/cXen_FoxA2_CR_rep2/cXen_FoxA2_CR_rep2_0_120.bowtie_tune.macs2_summits.bed.meme_5000_sample/cXen_FoxA2_CR_rep2_0_120.bowtie_tune.macs2_summits.bed.filtered_summits.padded_300.fa 1> $metrics
getsize ran successfully in 0.140448 seconds
Starting fasta-most: fasta-most -min 50 < /Extension_HDD1/Xen_PP/data/cXen_FoxA2_CR_rep2/cXen_FoxA2_CR_rep2_0_120.bowtie_tune.macs2_summits.bed.meme_5000_sample/cXen_FoxA2_CR_rep2_0_120.bowtie_tune.macs2_summits.bed.filtered_summits.padded_300.fa 1> $metrics
fasta-most ran successfully in 0.139545 seconds
Starting fasta-center: fasta-center -dna -len 100 < /Extension_HDD1/Xen_PP/data/cXen_FoxA2_CR_rep2/cXen_FoxA2_CR_rep2_0_120.bowtie_tune.macs2_summits.bed.meme_5000_sample/cXen_FoxA2_CR_rep2_0_120.bowtie_tune.macs2_summits.bed.filtered_summits.padded_300.fa 1> /Extension_HDD1/Xen_PP/data/cXen_FoxA2_CR_rep2/cXen_FoxA2_CR_rep2_0_120.bowtie_tune.macs2_summits.bed.meme_5000_sample/seqs-centered
fasta-center

CompletedProcess(args='meme-chip -oc /Extension_HDD1/Xen_PP/data/cXen_FoxA2_CR_rep2/cXen_FoxA2_CR_rep2_0_120.bowtie_tune.macs2_summits.bed.meme_5000_sample/ -streme-nmotifs 20 -meme-nmotifs 20 /Extension_HDD1/Xen_PP/data/cXen_FoxA2_CR_rep2/cXen_FoxA2_CR_rep2_0_120.bowtie_tune.macs2_summits.bed.filtered_summits.padded_300.fa -meme-p 10 -meme-searchsize 1500000', returncode=0)

In [ ]:
# CutMatrix

samtools view -b -h -f 3 -F 4 -F 8 -o outbam_dup.marked_Ctcf-0-120_sorted.bam dup.marked_Ctcf-0-120_sorted.bam

samtools index outbam_dup.marked_Ctcf-0-120_sorted.bam -@ 30

gff2bed < fimo_out_2/fimo.gff | awk 'BEGIN {IFS="\t"; OFS="\t";} {print $1,$2,$3,$4,$5,$6}' > fimo_out_2/fimo.bed


make_cut_matrix -v -b '(25-150 1)' -d -o 0 -r 100 -p 1 -f 3 -F 4 -F 8 -q 0 /Extension_HDD1/mouse_embryo_sc_rna_seq/mESC_PrE/hts.igb.uci.edu/paulap21061704/outbam_dup.marked_Ctcf-0-120_sorted.bam fimo_out_2/fimo.bed > fimo_out_2/fimo.cuts.freq.txt


#Count motif length 

tmp=`echo fimo_out_5 |cut -d "." -f3|wc -c`
mlen=$(( tmp - 1 ))


Rscript ./run_centipede_parker.R fimo_out_2/fimo.cuts.freq.txt fimo_out_2/fimo.bed fimo_out_2/fimo.png 10